In [2]:
from bs4 import BeautifulSoup as bs
import requests as req
import json, os, pprint, time, re
import pandas as pd
import numpy as np



### 先為了寫入檔案預先做準備

In [ ]:
folderPath = 'IMDB'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)
row = ["Title", "Author", "Date", "Up Vote", "Total Vote", "Rating", "Review"]

### 將爬蟲程式寫成FUNCTION 方便之後進行多部電影迴圈爬蟲

In [3]:
def get_IMDB_review(imdb_id):
    

    base_url = "https://www.imdb.com/"
    url=f"https://www.imdb.com/title/{imdb_id}/reviews?ref_=tt_ql_3"
    key = ""
    res = req.get(url)
    res.encoding = 'utf-8'
    soup = bs(res.text, "lxml")
    # 第一層我們要的欄位
    title = [item.select_one(".title").text for item in soup.select(".lister-item-content")]
    author = [item.select_one(".display-name-link").text for item in soup.select(".lister-item-content")]
    date = [item.select_one(".review-date").text for item in soup.select(".lister-item-content")]
    upvote = [item.select_one('.actions.text-muted').text.split(sep=" ")[20] for item in soup.select(".lister-item-content")]
    totalvote = [item.select_one('.actions.text-muted').text.split(sep=" ")[23] for item in soup.select(".lister-item-content")]
    rating=[(item.select_one("span.rating-other-user-rating > span").text if len(item.select("span.rating-other-user-rating > span"))==2 else "") for item in soup.select(".lister-item-content") ]
    review = [item.select_one('.text').text for item in soup.select(".lister-item-content")]

    # 第二波找LOADMORE資料
    load_more = soup.select_one(".load-more-data")
    flag = True
    # 找到load_more tag, 才需要往下抓
    # 第二波的第一次抓loadmore
    if load_more.text != '\n':
        ajaxurl = load_more['data-ajaxurl']
        base_url = base_url + ajaxurl + "?ref_=undefined&paginationKey="
        key = load_more['data-key']
    else :
        while flag:
            url = base_url + key
        #     print("url = ", url)
            res = req.get(url)
            res.encoding = 'utf-8'
            soup = bs(res.text, "lxml")
            title2 = [item.select_one(".title").text for item in soup.select(".lister-item-content")]
            title += title2
            author2 = [item.select_one(".display-name-link").text for item in soup.select(".lister-item-content")]
            author += author2
            date2 = [item.select_one(".review-date").text for item in soup.select(".lister-item-content")]
            date += date2
            upvote2 = [item.select_one('.actions.text-muted').text.split(sep=" ")[20] for item in soup.select(".lister-item-content")]
            upvote += upvote2
            totalvote2 = [item.select_one('.actions.text-muted').text.split(sep=" ")[23] for item in soup.select(".lister-item-content")]
            totalvote += totalvote2
            rating2 =[(item.select_one("span.rating-other-user-rating > span").text if len(item.select("span.rating-other-user-rating > span"))==2 else "") for item in soup.select(".lister-item-content") ]
            rating += rating2
            review2 = [item.select_one('.text').text for item in soup.select(".lister-item-content")]
            review += review2

            load_more = soup.select_one(".load-more-data")
            if load_more:
                key = load_more['data-key']
            else:
                flag = False
    sumlist = np.array((title, author, date, upvote, totalvote, rating, review))
    length = sumlist.T.shape[0] 
    IMDBID = np.full((length,1),imdb_id)
    convertnp = np.concatenate((IMDBID,sumlist.T),axis = 1)
    df = pd.DataFrame(convertnp,columns = ['IMDBId','title', 'author', 'date', 'upvote', 'totalvote', 'rating', 'review'])
#     df = pd.DataFrame(list(zip(title, author, date, upvote, totalvote, rating, review)), columns=row)
    df.to_csv(f"{folderPath}/{imdb_id}.csv",index = False)
#     print(df)
    print(f"{imdb_id} success")

In [4]:
# 讀取要爬蟲的檔案 如:無須清整檔案裡的IMDB評論
df1_TMDB = pd.read_json('ng_homework.json')

In [5]:
# 資料清整
# imdbID抓下來的有空值，也有NaN，先將空值轉為NaN
df1_TMDB['imdb_id'].replace('',np.NaN,inplace=True)
df1_TMDB['imdb_id'].replace('NaN',np.NaN,inplace=True)
# 取得需要的欄位做清整
df2=df1_TMDB[['imdb_id','en_title']] # 討論是否需要英文title?
# 將NaN都去除
df3=df2.dropna()
# 確認清整資料
df3.info()
# 抓取可用的IMDB ID 
imdb_id=[imdbID for imdbID in df3['imdb_id']]# if imdbID != NaN 前面已篩選 這邊不再做判斷
# 抓取IMDB title
# en_title=[title for title in df3['en_title'] ]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118765 entries, 36 to 860239
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   imdb_id   118765 non-null  object
 1   en_title  118765 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB


In [40]:
# in imdb_id如要分開爬蟲從x筆到y筆 則選取範圍 [get_IMDB_review(i) for i in imdb_id[x:y]]
[get_IMDB_review(i) for i in imdb_id[118741:118766]]
# [get_IMDB_review(i) for i in imdb_id]

# [get_IMDB_review(i) for i in imdb_id]
    
    
#     title_enreg_new=re.sub(r" |\r|\n|　|\s", '', en_title) # 這邊之後會直接使用豪哥清整完的正則過的標題 ，if標題過長解決

tt6610032 success
tt11815290 success
tt14115858 success
tt13038942 success
tt1446143 success
tt0033503 success
tt4202038 success
tt7386188 success
tt9783444 success
tt7085226 success
tt0247189 success
tt13032932 success
tt8030900 success
tt3835250 success
tt9849194 success
tt10993818 success
tt13813440 success
tt11197554 success
tt15409928 success
tt16763512 success
tt9042480 success
tt0008939 success
tt0008955 success
tt0008526 success
tt12524030 success
tt9855190 success
tt1474493 success
tt7707372 success
tt8433786 success
tt0422730 success
tt12825160 success
tt2923466 success
tt0024847 success
tt0460126 success
tt11633116 success
tt7208866 success
tt0010548 success
tt9755432 success
tt13021082 success
tt12710950 success
tt11415478 success
tt12092794 success
tt13010786 success
tt12141112 success
tt10521144 success
tt2353868 success
tt2262405 success
tt0005515 success
tt1415899 success
tt9001550 success
tt3433680 success
tt7131704 success
tt0389430 success
tt9820498 success
tt1608187

tt11947162 success
tt11947160 success
tt11947154 success
tt11947148 success
tt7909174 success
tt7909196 success
tt7909188 success
tt7909178 success
tt11394278 success
tt7909168 success
tt11750260 success
tt13333616 success
tt12881690 success
tt9363342 success
tt0319063 success
tt1494802 success
tt1869507 success
tt7054916 success
tt1695133 success
tt4761912 success
tt7604754 success
tt3060342 success
tt10980510 success
tt8562378 success
tt2360164 success
tt12193482 success
tt11269342 success
tt5157426 success
tt8327630 success
tt1300158 success
tt3906366 success
tt9000234 success
tt0829252 success
tt11473680 success
tt7830416 success
tt13085592 success
tt12549920 success
tt12396088 success
tt11468316 success
tt12885984 success
tt10303430 success
tt12165726 success
tt12390814 success
tt11961992 success
tt4898792 success
tt10725448 success
tt11585200 success
tt8804580 success
tt12877622 success
tt12877436 success
tt12618046 success
tt12867106 success
tt12882620 success
tt12885438 success

tt1058744 success
tt0150164 success
tt0142823 success
tt0142395 success
tt0142712 success
tt0142981 success
tt0142877 success
tt0142869 success
tt1047618 success
tt10469586 success
tt1058730 success
tt1058760 success
tt1043643 success
tt0018877 success
tt0019020 success
tt0018730 success
tt0019409 success
tt0019333 success
tt0019246 success
tt8055238 success
tt9525214 success
tt12015668 success
tt1570460 success
tt12757930 success
tt12343322 success
tt10099700 success
tt12750140 success
tt12745164 success
tt12754866 success
tt12070904 success
tt12816634 success
tt11628318 success
tt2818666 success
tt7681186 success
tt12723524 success
tt13341400 success
tt7184122 success
tt1575530 success
tt12722610 success
tt3679164 success
tt12627080 success
tt12047822 success
tt12690904 success
tt6494462 success
tt10133638 success
tt4426166 success
tt0039622 success
tt9496480 success
tt2388516 success
tt1414803 success
tt0442109 success
tt0222479 success
tt3915668 success
tt0003618 success
tt0152478 

tt6543788 success
tt12569538 success
tt0011278 success
tt1316434 success
tt7093902 success
tt7724770 success
tt6256836 success
tt0353263 success
tt0295890 success
tt3415270 success
tt8803574 success
tt9051858 success
tt6886578 success
tt6760884 success
tt6459962 success
tt9208300 success
tt10399616 success
tt12192884 success
tt6352568 success
tt11027932 success
tt12576394 success
tt13138770 success
tt10098448 success
tt14191840 success
tt1047485 success
tt4600982 success
tt12854600 success
tt12898406 success
tt0383984 success
tt10254706 success
tt0300653 success
tt11107752 success
tt2327645 success
tt4516040 success
tt9116234 success
tt0403385 success
tt6985436 success
tt0428126 success
tt0007724 success
tt0402949 success
tt10957044 success
tt0147132 success
tt0365489 success
tt11220408 success
tt10126040 success
tt9398530 success
tt12317568 success
tt0488420 success
tt3278540 success
tt12588160 success
tt12584954 success
tt0175522 success
tt4645032 success
tt12560818 success
tt7122050

tt4651298 success
tt10328430 success
tt11904798 success
tt11905696 success
tt12752790 success
tt1625198 success
tt8762210 success
tt11585996 success
tt7466496 success
tt1353841 success
tt4097926 success
tt11233544 success
tt0203498 success
tt6021628 success
tt11166720 success
tt11984754 success
tt3551378 success
tt7935526 success
tt11105790 success
tt7281114 success
tt12063208 success
tt6432158 success
tt11973904 success
tt2076202 success
tt11343416 success
tt0265069 success
tt1008956 success
tt6780790 success
tt12172388 success
tt14551124 success
tt0345335 success
tt11629124 success
tt6483422 success
tt12264582 success
tt8986276 success
tt8385384 success
tt0011781 success
tt0847113 success
tt12861212 success
tt11867738 success
tt12416276 success
tt12422140 success
tt0081234 success
tt12416400 success
tt12420100 success
tt12433896 success
tt12433928 success
tt12433950 success
tt8506500 success
tt12413186 success
tt12316704 success
tt9023894 success
tt9764152 success
tt10944596 success


tt1106755 success
tt5432132 success
tt0443889 success
tt3864730 success
tt0358253 success
tt1090681 success
tt1511492 success
tt5889288 success
tt2501404 success
tt4789732 success
tt2866156 success
tt0444465 success
tt2104992 success
tt3415220 success
tt4667664 success
tt3756730 success
tt1142573 success
tt6720682 success
tt3864872 success
tt2104951 success
tt12293938 success
tt11756010 success
tt4319244 success
tt4723200 success
tt8770120 success
tt3576334 success
tt4509610 success
tt2320770 success
tt4026538 success
tt2400395 success
tt1571577 success
tt3212368 success
tt1830743 success
tt1556940 success
tt2436164 success
tt3336716 success
tt3799592 success
tt2400459 success
tt2153470 success
tt2596292 success
tt3157528 success
tt0450756 success
tt2783514 success
tt0161464 success
tt3853680 success
tt0014051 success
tt2956596 success
tt0005675 success
tt0007030 success
tt0018900 success
tt0006907 success
tt0012069 success
tt0007603 success
tt4630326 success
tt0007450 success
tt000691

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [9]:
imdb_id[95815:95825]

['tt0166100',
 'tt8422212',
 'tt12104426',
 'tt0127163',
 'tt7823908',
 'tt10714348',
 'tt14666508',
 'tt14588478',
 'tt14271284',
 'tt0207636']

In [22]:
tt4861980、tt4286668、tt1208671、tt4106286、tt0473493、tt0949757、tt6520712、tt6063364、tt8521926、tt2150131、tt7823908

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6708 entries, 306 to 856784
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   imdb_id   6708 non-null   object
 1   en_title  6708 non-null   object
dtypes: object(2)
memory usage: 157.2+ KB


6708

tt14022314 success
tt12057400 success
tt12580336 success
tt13309416 success
tt22487726 success
tt14294638 success
tt14242710 success
tt14297960 success
tt6423766 success
tt14414730 success
tt22867144 success
tt23901570 success
tt16392540 success
tt23744586 success
tt23332322 success
tt21478814 success
tt0094961 success
tt0095756 success
tt0117104 success
tt0120836 success
tt0247745 success
tt0024264 success
tt0091671 success
tt0049452 success
tt1185344 success
tt0808526 success
tt0842926 success
tt0119843 success
tt0054197 success
tt1287468 success
tt0286162 success
tt1598778 success
tt1245526 success
tt1092026 success
tt1028576 success
tt0111187 success
tt0970866 success
tt0107079 success
tt0041650 success
tt0106521 success
tt0087344 success
tt0433035 success
tt0091668 success
tt1421051 success
tt0063592 success
tt0073043 success
tt0271219 success
tt0023948 success
tt0092627 success
tt0093743 success
tt1247662 success
tt1399683 success
tt0097987 success
tt0076740 success
tt0089997 suc

tt0963966 success
tt0034746 success
tt1106860 success
tt0086346 success
tt0339419 success
tt0081059 success
tt0094321 success
tt0096203 success
tt0268200 success
tt0091637 success
tt0116905 success
tt0043503 success
tt0332712 success
tt1232783 success
tt0091707 success
tt0105017 success
tt0037611 success
tt0234829 success
tt0368774 success
tt0090856 success
tt0088146 success
tt0064100 success
tt0095925 success
tt0056860 success
tt0085672 success
tt1186830 success
tt1210042 success
tt1462758 success
tt0095107 success
tt0110399 success
tt0109417 success
tt1127896 success
tt0119937 success
tt0102898 success
tt0060490 success
tt0092857 success
tt0109297 success
tt0033028 success
tt0044760 success
tt0780511 success
tt0166943 success
tt0101746 success
tt1407050 success
tt0062138 success
tt1188996 success
tt1068678 success
tt0047947 success
tt0099018 success
tt0041239 success
tt0285728 success
tt0050629 success
tt0929632 success
tt0433398 success
tt0075909 success
tt0106557 success
tt0038854 

tt1041829 success
tt1259571 success
tt0054692 success
tt0472259 success
tt0115678 success
tt1018818 success
tt0085154 success
tt0090583 success
tt0417385 success
tt0457400 success
tt0488658 success
tt1114677 success
tt0443450 success
tt0176783 success
tt0396707 success
tt0048261 success
tt0110329 success
tt0090327 success
tt1067106 success
tt0102011 success
tt0118744 success
tt1083456 success
tt0434424 success
tt0156812 success
tt0090021 success
tt0118691 success
tt0317132 success
tt0033226 success
tt0120133 success
tt0119484 success
tt0044030 success
tt0318283 success
tt0109838 success
tt0298296 success
tt0131704 success
tt0250310 success
tt0120620 success
tt0085750 success
tt0076070 success
tt1103982 success
tt1136608 success
tt0037120 success
tt1045778 success
tt0111323 success
tt0983193 success
tt1231277 success
tt0105699 success
tt0942384 success
tt0102388 success
tt0368226 success
tt0099385 success
tt0082894 success
tt0078718 success
tt0865554 success
tt1182345 success
tt0086192 

tt0091369 success
tt1046163 success
tt1091751 success
tt1477715 success
tt0315297 success
tt0103125 success
tt0086320 success
tt0028010 success
tt0095082 success
tt0195778 success
tt0323944 success
tt0269095 success
tt0210382 success
tt0058947 success
tt1135985 success
tt0790686 success
tt0093560 success
tt0489037 success
tt0484111 success
tt0443295 success
tt0331933 success
tt0303933 success
tt0244000 success
tt0997047 success
tt0494652 success
tt0492619 success
tt0211933 success
tt1185416 success
tt0335559 success
tt0796366 success
tt0804516 success
tt0132477 success
tt0483607 success
tt0465580 success
tt0808151 success
tt0168786 success
tt0390022 success
tt0328099 success
tt0290332 success
tt1034331 success
tt1129442 success
tt0396652 success
tt0310281 success
tt0262396 success
tt0854678 success
tt1091617 success
tt0091445 success
tt0083929 success
tt0408524 success
tt0094138 success
tt0481536 success
tt0988047 success
tt0072856 success
tt0947802 success
tt0417433 success
tt0845046 

tt0131325 success
tt0096794 success
tt0301470 success
tt0055824 success
tt0371739 success
tt0098622 success
tt0089346 success
tt0081554 success
tt0085407 success
tt0081353 success
tt0313542 success
tt0077928 success
tt1130884 success
tt1130080 success
tt1152836 success
tt0814314 success
tt0076618 success
tt0252501 success
tt0109439 success
tt0099740 success
tt0084917 success
tt0116259 success
tt0097940 success
tt0066808 success
tt0082533 success
tt0086969 success
tt0070510 success
tt0104694 success
tt0082136 success
tt0094890 success
tt0101917 success
tt0396752 success
tt0366551 success
tt0082495 success
tt0765476 success
tt0104231 success
tt0120696 success
tt0411477 success
tt0155975 success
tt0387575 success
tt0356470 success
tt0107563 success
tt0035423 success
tt0110657 success
tt0116040 success
tt0042332 success
tt0086955 success
tt0048750 success
tt0331953 success
tt0092605 success
tt0109190 success
tt0053580 success
tt0324554 success
tt0062467 success
tt0066206 success
tt0095690 

...

tt3830222 success
tt3741834 success
tt4034228 success
tt3553976 success
tt1800302 success
tt4527664 success
tt2368619 success
tt2822672 success
tt4183692 success
tt2404435 success
tt2436682 success
tt2091935 success
tt1366338 success
tt1179933 success
tt2279339 success
tt1677720 success
tt1620935 success
tt4052882 success
tt1517451 success
tt4624424 success
tt1226766 success
tt3281548 success
tt2387499 success
tt3268668 success
tt1655603 success
tt3778644 success
tt4573516 success
tt2776878 success
tt4560436 success
tt4824308 success
tt4034354 success
tt3387266 success
tt3829266 success
tt3181776 success
tt3631112 success
tt3717252 success
tt4468740 success
tt1396484 success
tt4779682 success
tt2872518 success
tt4486986 success
tt3401882 success
tt4682786 success
tt4481514 success
tt4786282 success
tt3766354 success
tt4501244 success
tt2380307 success
tt1966359 success
tt4776998 success
tt2404297 success
tt4086018 success
tt4765284 success
tt1412528 success
tt1648190 success
tt2283362 

tt1583421 success
tt1397514 success
tt0043744 success
tt0485985 success
tt1656190 success
tt0763831 success
tt1222815 success
tt1611224 success
tt1630036 success
tt1195478 success
tt1430626 success
tt0816711 success
tt1637725 success
tt0096953 success
tt0144637 success
tt0093029 success
tt0810913 success
tt1462769 success
tt1242460 success
tt0099816 success
tt1855325 success
tt1071875 success
tt1714203 success
tt1605630 success
tt1093357 success
tt1446714 success
tt0026071 success
tt1742650 success
tt0120244 success
tt0105291 success
tt1107319 success
tt1748197 success
tt1674784 success
tt1486192 success
tt1506999 success
tt0005078 success
tt1392170 success
tt1308729 success
tt0085891 success
tt1716772 success
tt1462041 success
tt0035749 success
tt0119349 success
tt1068242 success
tt1491044 success
tt1121096 success
tt0803096 success
tt1024648 success
tt0490215 success
tt1623205 success
tt1924429 success
tt1663662 success
tt1535108 success
tt1560747 success
tt1300854 success
tt1853728 

tt0027948 success
tt0029604 success
tt0030386 success
tt0029852 success
tt0033238 success
tt0032206 success
tt1270296 success
tt0110391 success
tt0024865 success
tt0339727 success
tt0281680 success
tt0038213 success
tt1193507 success
tt0023926 success
tt0024548 success
tt0464154 success
tt0115645 success
tt1452628 success
tt0036628 success
tt1458175 success
tt0036422 success
tt0036984 success
tt0038704 success
tt0040594 success
tt0040745 success
tt0046248 success
tt0758752 success
tt0035959 success
tt0033729 success
tt0049233 success
tt0050530 success
tt0034458 success
tt0052969 success
tt1289401 success
tt0052543 success
tt0054469 success
tt0053571 success
tt0056556 success
tt1695994 success
tt1341188 success
tt0035675 success
tt0021746 success
tt0021730 success
tt0120769 success
tt0043044 success
tt1504319 success
tt0062038 success
tt1564585 success
tt0052365 success
tt0062883 success
tt0063483 success
tt0062974 success
tt0065215 success
tt0065054 success
tt0019237 success
tt0107413 

tt2850386 success
tt7938472 success
tt5109280 success
tt6472976 success
tt5108870 success
tt8085790 success
tt7545266 success
tt6394270 success
tt9032400 success
tt7737786 success
tt8235660 success
tt6107548 success
tt1206885 success
tt8385148 success
tt1478839 success
tt5886046 success
tt8367814 success
tt8376234 success
tt4916630 success
tt6924650 success
tt8356942 success
tt6977338 success
tt8350360 success
tt6439020 success
tt8332922 success
tt2627434 success
tt9585282 success
tt5467644 success
tt10039344 success
tt9735328 success
tt9876312 success
tt9876308 success
tt9734574 success
tt9734988 success
tt9735052 success
tt9735072 success
tt9735090 success
tt9735110 success
tt9588290 success
tt9588282 success
tt9588218 success
tt9588116 success
tt9651152 success
tt9650436 success
tt9585340 success
tt9585030 success
tt9587716 success
tt8088800 success
tt8057394 success
tt7990426 success
tt8023204 success
tt8020970 success
tt9011054 success
tt9011110 success
tt9011134 success
tt9011222

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,